<a href="https://colab.research.google.com/github/blackdragon18/codificacion/blob/main/car_collision_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 541-B Assignment 1(B):  Car collision

#### Name: Ishaan Chawathe
#### Stevens ID: 20031207

You have been given a dataset containing the positions of two
cars labeled as x1 and x2, along with a ground truth indicator y. In this context,
y equals 0 if a collision occurs and 1 if there is no collision. Your task is to create
a neural network capable of predicting collisions from scratch, without relying
on pre-existing modules.

In [1]:
"""
    Download the datset from the below link
"""
!wget 'https://raw.githubusercontent.com/liususan091219/cs541/main/hw1/prob2/dataset_1/train.csv'
!wget 'https://raw.githubusercontent.com/liususan091219/cs541/main/hw1/prob2/dataset_1/test.csv'

--2025-02-23 19:40:44--  https://raw.githubusercontent.com/liususan091219/cs541/main/hw1/prob2/dataset_1/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48252 (47K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  47.12K  --.-KB/s    in 0.01s   

2025-02-23 19:40:44 (3.87 MB/s) - ‘train.csv’ saved [48252/48252]

--2025-02-23 19:40:44--  https://raw.githubusercontent.com/liususan091219/cs541/main/hw1/prob2/dataset_1/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20695 (20K) [t

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
# Here, we are viewing more information about the dataset
print("Training Data Info:")
print(df_train.info())
print(df_train.describe())

print("\nTest Data Info:")
print(df_test.info())
print(df_test.describe())

Training Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      3500 non-null   float64
 1   x2      3500 non-null   float64
 2   y       3500 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 82.2 KB
None
                x1           x2            y
count  3500.000000  3500.000000  3500.000000
mean      2.466987     2.542629     0.823429
std       1.447333     1.445768     0.381360
min       0.001000     0.001000     0.000000
25%       1.194250     1.290250     1.000000
50%       2.515500     2.598500     1.000000
75%       3.719000     3.784750     1.000000
max       5.000000     5.000000     1.000000

Test Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      1500 non-null   fl

In [4]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Extracting the features and label 'y'
X_train = df_train.iloc[:, 0:2].values.astype('float32')
y_train = df_train['y'].values.astype('float32')

X_test = df_test.iloc[:, 0:2].values.astype('float32')
y_test = df_test['y'].values.astype('float32')

# Now, we are Standardizing the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Viewing the shapes of Test and train data
print("Shapes of training data:")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)

print("\nShapes of test data:")
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)


Shapes of training data:
X_train: (3500, 2)
y_train: (3500,)

Shapes of test data:
X_test: (1500, 2)
y_test: (1500,)


##Part-1 [15 pts]

Implement a class of model, where in the init function, you can initialize the parameters of trainable weights like v11, v12, v21, v22, v31, v32, w1, w2. You can employ PyTorch to initialize them randomly.



In [5]:
import torch
import torch.nn.functional as F

seed = 42
torch.manual_seed(seed)

class Car_collision:
    def __init__(self):

        #In our Neural Netwok Architeture, we are initializing the paramters of trainable weights
        # For each hidden neuron, we have two weights (for x1 and x2) and one bias.

        #This is the neuron 1 in Hidden layer
        self.v11 = torch.randn(1, requires_grad=True)
        self.v21 = torch.randn(1, requires_grad=True)
        self.v31 = torch.randn(1, requires_grad=True)

        #This is the neuron 2 in Hidden layer
        self.v12 = torch.randn(1, requires_grad=True)
        self.v22 = torch.randn(1, requires_grad=True)
        self.v32 = torch.randn(1, requires_grad=True)

        # Output layer weights from the two hidden neurons:
        self.w1 = torch.randn(1, requires_grad=True)
        self.w2 = torch.randn(1, requires_grad=True)

    def forward(self, x):
        # For the Forward pass, I have used x: a torch tensor of shape (2,)

        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32)

        # Below are the Hidden layer computations using ReLU acttivation function:
        h1_linear = x[0] * self.v11 + x[1] * self.v21 + self.v31
        h2_linear = x[0] * self.v12 + x[1] * self.v22 + self.v32
        h1 = torch.relu(h1_linear)
        h2 = torch.relu(h2_linear)

        # Output layer is designed such that it calculaes a weighted sum followed by sigmoid activation function.
        output_linear = h1 * self.w1 + h2 * self.w2
        y_pred = torch.sigmoid(output_linear)
        return y_pred

    def parameters_list(self):
        # Returning the list of all parameters for updating manually
        return [self.v11, self.v21, self.v31, self.v12, self.v22, self.v32, self.w1, self.w2]

model = Car_collision()


##Part-2 [15 pts]
Implement a training loop that iterates over all the data, computes the gradient scores, updates all weights defined in the Car_collision model, and calculates the average loss obtained from all iterations. You can choose best learning rate, and select epochs number

In [6]:
import torch
import torch.nn.functional as F

#In the training loop, I have introduced momentum and velocity.
#For every parameter, we maintain a "velocity" that accumulates past gradient information.
#So, Instead of updating the parameter solely based on the current gradient, we update it using a running average of previous gradients. This accumulated value is called the velocity.
#The momentum term determines how much of the previous velocity is retained in the current update.
#By using momentum, the model remembers the direction of past updates. This helps it to move faster in correct gradient directions and also reduces oscillations.
#Hence, this leads to faster convergence and a more stable training process.

#By using these concepts, I have observed that there is a noticable increase in the accuracy value

# Preparing the training samples
train_samples = list(zip(X_train, y_train))

learning_rate = 2e-4
num_epochs = 30
momentum = 0.9

# Initializing velocity for each parameter
velocities = [torch.zeros_like(param) for param in model.parameters_list()]

for epoch in range(num_epochs):
    total_loss = 0.0
    for x_sample, y_true in train_samples:
        x_tensor = torch.tensor(x_sample, dtype=torch.float32)

        #forward pass
        y_pred = model.forward(x_tensor)
        y_true_tensor = torch.tensor([y_true], dtype=torch.float32)

        # Calculating the binary cross entropy loss
        loss = F.binary_cross_entropy(y_pred, y_true_tensor)

        # Backward pass: calculating the gradients
        loss.backward()

        # Updating parameters with momentum
        with torch.no_grad():
            for i, param in enumerate(model.parameters_list()):
                velocities[i] = momentum * velocities[i] - learning_rate * param.grad
                param.add_(velocities[i])
                param.grad.zero_()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_samples)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")


Epoch 1/30, Loss: 0.4971
Epoch 2/30, Loss: 0.4679
Epoch 3/30, Loss: 0.4637
Epoch 4/30, Loss: 0.4591
Epoch 5/30, Loss: 0.4534
Epoch 6/30, Loss: 0.4474
Epoch 7/30, Loss: 0.4421
Epoch 8/30, Loss: 0.4380
Epoch 9/30, Loss: 0.4349
Epoch 10/30, Loss: 0.4328
Epoch 11/30, Loss: 0.4314
Epoch 12/30, Loss: 0.4306
Epoch 13/30, Loss: 0.4300
Epoch 14/30, Loss: 0.4297
Epoch 15/30, Loss: 0.4295
Epoch 16/30, Loss: 0.4294
Epoch 17/30, Loss: 0.4294
Epoch 18/30, Loss: 0.4293
Epoch 19/30, Loss: 0.4293
Epoch 20/30, Loss: 0.4292
Epoch 21/30, Loss: 0.4292
Epoch 22/30, Loss: 0.4291
Epoch 23/30, Loss: 0.4290
Epoch 24/30, Loss: 0.4289
Epoch 25/30, Loss: 0.4288
Epoch 26/30, Loss: 0.4288
Epoch 27/30, Loss: 0.4288
Epoch 28/30, Loss: 0.4287
Epoch 29/30, Loss: 0.4287
Epoch 30/30, Loss: 0.4287


##Part-4 [5 pts]
Implement a loop that iterates over all the test data, predicts scores, and assigns a value of 1 if the score is greater than 0.5, otherwise 0.


Then calculate the test accuracy

In [7]:
# Preparing the test samples
test_samples = list(zip(X_test, y_test))

with torch.no_grad():
    correct = 0
    total = 0
    for x_sample, y_true in test_samples:
        x_tensor = torch.tensor(x_sample, dtype=torch.float32)
        #predicting the value
        y_pred = model.forward(x_tensor)
        # As per our condition, if the predicted label value is greater than threshold 0.5, there is no collision
        pred_label = 1 if y_pred.item() > 0.5 else 0
        if pred_label == int(y_true):
            correct += 1
        total += 1
#Calculating the Test accuracy
accuracy = (correct / total) * 100
print(f"Accuracy on test data: {accuracy:.2f}%")


Accuracy on test data: 81.67%


# **Accuracy report:**
<pre>
            Instructor's Accuracy     My Accuracy
Dataset 1:      77.07%                 81.67%

Dataset 2:      67.40%                 68.47%

Dataset 3:      61.87%                 63.20%

Dataset 4:      79.07%                 82.00%

Dataset 5:      75.53%                 77.67%

</pre>